In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# Loading the cleaned dataset, in which within-mouse outliers have been removed and then the MEDIAN per mouse has been calculated
# Outlier biological replicates are still present.  Some mice are missing data for some proteins.
# (8/76 proteins are missing at least one data point.)
data = pd.read_csv('Data_Cortex_Nuclear_Means_Clean.csv')
data.replace(to_replace={'C/S': 'Learning', 'S/C': 'Control'}, inplace=True)
data.head()

,Genotype,Behavior,Treatment,class,Mouse,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,...,pGSK3B_Tyr216_N,SHH_N,BAD_N,BCL2_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N
0,Control,Learning,Memantine,c-CS-m,309,0.421063,0.576510,0.356447,2.278252,4.354744,...,0.832559,0.192871,0.151524,NaN,0.115433,0.423088,0.136374,0.169300,0.169590,1.674160
1,Control,Learning,Memantine,c-CS-m,311,0.600591,0.678479,0.334548,2.277561,4.234706,...,0.673229,0.188092,0.166315,0.168952,NaN,0.426774,0.141512,0.201386,0.200284,1.193285
2,Control,Learning,Memantine,c-CS-m,320,0.547676,0.796012,0.398649,2.664515,5.498960,...,0.859745,0.218300,0.159284,0.138537,0.123580,0.538909,0.141971,0.187006,NaN,1.624791
3,Control,Learning,Memantine,c-CS-m,321,0.438728,0.652045,0.359441,2.686405,5.298355,...,0.922949,0.238123,NaN,0.158817,NaN,0.559531,0.150542,0.220306,NaN,1.678225
4,Control,Learning,Memantine,c-CS-m,322,0.358931,0.512223,0.315105,2.111664,4.231444,...,0.828905,0.239495,NaN,0.154224,0.141323,0.493076,NaN,NaN,NaN,1.607234


In [ ]:
from sklearn.